In [15]:
import pandas as pd

companies = pd.read_csv("companies.txt", delimiter="\t", encoding='ISO-8859-1')
round2 = pd.read_csv("rounds2.csv", encoding='ISO-8859-1')

companies['permalink'] = companies['permalink'].str.lower()
round2['company_permalink'] = round2['company_permalink'].str.lower()
round2['raised_amount_usd'].fillna(0, inplace=True)

# Get details of unique companies in companies file
print("Unique values in companies DataFrame :", companies['permalink'].nunique())

# Get details of unique companies in round2
print("Unique values in companies round2 :", round2['company_permalink'].nunique())

# Find out of there are companies in round2 that are not present in companies file

df1 = pd.DataFrame(companies,columns=['permalink'])
df2 = pd.DataFrame(round2,columns=['company_permalink'])

set_diff_df = pd.concat([df2, df1, df1], sort=True).drop_duplicates(keep=False)
print("Additional companies in round2 that does not exist in companies DataFrame :", set_diff_df['company_permalink'].count())

# Merger two data frames
master_frame = pd.merge(companies, round2, how='outer', left_on=['permalink'], right_on=['company_permalink'])

master_frame.describe(include='all')


Unique values in companies DataFrame : 66368
Unique values in companies round2 : 66370
Additional companies in round2 that does not exist in companies DataFrame : 42493


,permalink,name,homepage_url,category_list,status,country_code,state_code,region,city,founded_at,company_permalink,funding_round_permalink,funding_round_type,funding_round_code,funded_at,raised_amount_usd
count,114947,114946,108813,111538,114947,106271,104003,104782,104785,94426,114949,114949,114949,31140,114949,1.149490e+05
unique,66368,66102,61191,27296,4,137,311,1092,5111,3978,66370,114949,14,8,5033,NaN
top,/organization/solarflare,Solarflare Communications,http://www.solarflare.com,Biotechnology,operating,USA,CA,SF Bay Area,San Francisco,01-01-2012,/organization/solarflare,/funding-round/b75ffc6e1cdd2a7b592b56179582e659,venture,A,01-01-2014,NaN
freq,19,19,19,8165,90152,72308,26664,19078,7422,4642,19,1,55494,14747,779,NaN
mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.613603e+06
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.044356e+08
min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000e+00
25%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.000000e+04
50%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000000e+06
75%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.000000e+06


In [1]:
%config IPCompleter.greedy = True